In [2]:
from osgeo import gdal
from PIL import Image
import os
import numpy as np
import keras.backend as K
import png
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
from __future__ import division, print_function

from keras.layers import Input, Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D, Cropping2D, Concatenate
from keras.layers import Lambda, Activation, BatchNormalization, Dropout
from keras.models import Model
from keras import backend as K


def downsampling_block(input_tensor, filters, padding='valid',  #下采样部分
                       batchnorm=False, dropout=0.0):
    _, height, width, _ = K.int_shape(input_tensor)
#     assert height % 2 == 0
#     assert width % 2 == 0

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(input_tensor)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    return MaxPooling2D(pool_size=(2,2))(x), x   #返回的是池化后的值和dropout后的值，这里dropout后的值用于上采样特征级联

def upsampling_block(input_tensor, skip_tensor, filters, padding='valid',
                     batchnorm=False, dropout=0.0):    #下采样部分
    x = Conv2DTranspose(filters, kernel_size=(2,2), strides=(2,2))(input_tensor)
    _, x_height, x_width, _ = K.int_shape(x)
    _, s_height, s_width, _ = K.int_shape(skip_tensor)
    h_crop = s_height - x_height
    w_crop = s_width - x_width
    assert h_crop >= 0
    assert w_crop >= 0
    if h_crop == 0 and w_crop == 0:
        y = skip_tensor
    else:                       #使级联时像素大小一致
        cropping = ((h_crop//2, h_crop - h_crop//2), (w_crop//2, w_crop - w_crop//2))
        y = Cropping2D(cropping=cropping)(skip_tensor)

    x = Concatenate()([x, y])         #特征级联

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    x = Conv2D(filters, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    return x                   #返回dropout后的值

def unet(height, width, channels, classes, features=64, depth=4,
         temperature=1.0, padding='valid', batchnorm=False, dropout=0.0):  #使用4个深度长的网络就是官网的典型网络
    x = Input(shape=(height, width, channels))
    inputs = x

    skips = []                   #用于存放下采样中，每个深度后，dropout后的值，以供之后级联使用
    for i in range(depth):
        x, x0 = downsampling_block(x, features, padding,
                                   batchnorm, dropout)
        skips.append(x0)
        features *= 2            #下采样过程中，每个深度往下，特征翻倍，即每次使用翻倍数目的滤波器

    x = Conv2D(filters=features, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    x = Conv2D(filters=features, kernel_size=(3,3), padding=padding)(x)
    x = BatchNormalization()(x) if batchnorm else x
    x = Activation('relu')(x)
    x = Dropout(dropout)(x) if dropout > 0 else x

    for i in reversed(range(depth)):    #下采样过程中，深度从深到浅
        features //= 2                  #每个深度往上。特征减少一倍
        x = upsampling_block(x, skips[i], features, padding,
                             batchnorm, dropout)

    x = Conv2D(filters=classes, kernel_size=(1,1))(x)

    logits = Lambda(lambda z: z/temperature)(x)      #简单的对x做一个变换
    probabilities = Activation('softmax')(logits)    #对输出的两类做softmax，转换为概率。形式如【0.1,0.9],则预测为第二类的概率更大。

    return Model(inputs=inputs, outputs=probabilities)

In [4]:
# def unet(img_w, img_h, channels, n_label):
#     inputs = Input((img_w, img_h, channels))
 
#     conv1 = Conv2D(8, (3, 3), activation="relu", padding="same")(inputs)
#     conv1 = Conv2D(8, (3, 3), activation="relu", padding="same")(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
#     conv2 = Conv2D(16, (3, 3), activation="relu", padding="same")(pool1)
#     conv2 = Conv2D(16, (3, 3), activation="relu", padding="same")(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
 
#     conv3 = Conv2D(32, (3, 3), activation="relu", padding="same")(pool2)
#     conv3 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
 
#     conv4 = Conv2D(64, (3, 3), activation="relu", padding="same")(pool3)
#     conv4 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv4)
#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
 
#     conv5 = Conv2D(128, (3, 3), activation="relu", padding="same")(pool4)
#     conv5 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv5)
 
#     up6 = Concatenate()([UpSampling2D(size=(2, 2))(conv5), conv4])
#     conv6 = Conv2D(64, (3, 3), activation="relu", padding="same")(up6)
#     conv6 = Conv2D(63, (3, 3), activation="relu", padding="same")(conv6)
 
#     up7 = Concatenate()([UpSampling2D(size=(2, 2))(conv6), conv3])
#     conv7 = Conv2D(32, (3, 3), activation="relu", padding="same")(up7)
#     conv7 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv7)
 
#     up8 = Concatenate()([UpSampling2D(size=(2, 2))(conv7), conv2])
#     conv8 = Conv2D(16, (3, 3), activation="relu", padding="same")(up8)
#     conv8 = Conv2D(16, (3, 3), activation="relu", padding="same")(conv8)
 
#     up9 = Concatenate()([UpSampling2D(size=(2, 2))(conv8), conv1])
#     conv9 = Conv2D(8, (3, 3), activation="relu", padding="same")(up9)
#     conv9 = Conv2D(8, (3, 3), activation="relu", padding="same")(conv9)

# #     conv10 = Conv2D(n_label, (1, 1), activation="sigmoid")(conv9)
#     conv10 = Conv2D(n_label, (1, 1), activation="softmax")(conv9)
 
#     model = Model(inputs=inputs, outputs=conv10)
#     model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

#     return model

In [5]:
model = unet(768, 768, 3, 4, features=8, depth=4, padding='same', batchnorm=True, dropout=0)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 768, 768, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 768, 768, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 768, 768, 8)  32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 768, 768, 8)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

activation_11 (Activation)      (None, 96, 96, 64)   0           batch_normalization_11[0][0]     
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 96, 96, 64)   36928       activation_11[0][0]              
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 96, 96, 64)   256         conv2d_12[0][0]                  
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 96, 96, 64)   0           batch_normalization_12[0][0]     
__________________________________________________________________________________________________
conv2d_transpose_2 (Conv2DTrans (None, 192, 192, 32) 8224        activation_12[0][0]              
__________________________________________________________________________________________________
concatenat

In [7]:
model.load_weights('unet_epoch17_valloss_0.5152_valacc_0.7979.h5')

In [8]:
img3path = "D:/kaggle/nongye/jingwei_round1_test_a_20190619/jingwei_round1_test_a_20190619/image_3.png"
img4path = "D:/kaggle/nongye/jingwei_round1_test_a_20190619/jingwei_round1_test_a_20190619/image_4.png"

In [9]:
def predict(imgpath, model, output_name='tmp.bmp', blocksize=768, step=768):
    
    ds = gdal.Open(imgpath)
    wx = ds.RasterXSize # 37241
    wy = ds.RasterYSize # 19903
    od = np.zeros((wy, wx), np.uint8)
    
    print('-----starting prediction-------')
    for cy in tqdm(range(step, wy, step)):
        for cx in range(step, wx, step):
            im = ds.ReadAsArray(cx-step, cy-step, blocksize, blocksize)[0:3, :, :].transpose(1, 2, 0)
            if (im.sum()==0): continue
            im =np.expand_dims(im, axis=0)
            r = model.predict(im/255, verbose=2)
            r = np.argmax(r, -1).reshape(blocksize, blocksize)
            od[cy-step:cy-step+blocksize, cx-step:cx-step+blocksize] = r
    print('----starting saving result-----')    
    w = png.Writer(wx, wy, bitdepth=2,greyscale=True)
    of = open('./predict/' +  output_name, 'wb')
    w.write_array(of, od.flat)
    of.close()

In [10]:
predict(img3path, model, output_name='image_3_predict.png', blocksize=768, step=768)

-----starting prediction-------


100%|██████████████████████████████████████████| 25/25 [17:00<00:00, 42.91s/it]


----starting saving result-----


In [11]:
predict(img4path, model, output_name='image_4_predict.png', blocksize=768, step=768)

-----starting prediction-------


100%|██████████████████████████████████████████| 37/37 [16:42<00:00, 20.65s/it]


----starting saving result-----
